# Imports and environment setup

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from PIL import ImageGrab
import cv2
import time
from mss import mss

#mouse control and window dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 1024
MONITOR_WIDTH = 800
MONITOR_HEIGHT = 640
import win32api, win32con
#Player Control
click_cooldown = 0.5

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from utils import label_map_util
from utils import visualization_utils as vis_util

# Load Model

In [ ]:
#Select model in the same folder as this script
MODEL_NAME = 'LeagueAI_v3'
MODEL_FILE = MODEL_NAME + '.tar.gz'
#DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'LeagueAI_v2.pbtxt')

NUM_CLASSES = 3

## Load the frozen model into memory

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
## Other

In [ ]:
## Champion helper classes
# click a postion in the screen using x,y coordinates
def click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)
# find the player character position marked as x,y (converting the boxes to a single point position)
def find_box_xy(box):
    playerwidth =  (box[3] - box[1])
    playerheight = box[2] - box[0]
    playerpos_y = (box[0] + playerheight/1.3)
    playerpos_x = (box[1] + playerwidth/2)
    return([playerpos_y, playerpos_x, playerheight, playerwidth])
# find out in which state on object is
def find_object_state(grid, minion_origin, player_origin):
    x_object = (minion_origin[0]/MONITOR_WIDTH)*SCREEN_WIDTH
    y_object = (minion_origin[1]/MONITOR_HEIGHT)*SCREEN_HEIGHT
    x_player = (player_origin[0]/MONITOR_WIDTH)*SCREEN_WIDTH
    y_player = (player_origin[1]/MONITOR_HEIGHT)*SCREEN_HEIGHT
    print(x_player)
    #TODO find out in which state an enemy minion is and return the state
    state_x = 1
    state_y = 1
    return([state_x, state_y])
# click a certain state given by x and y coordinate in the state grid
def click_state(state_x, state_y, state_width, state_height, player_origin):
    x_click = (player_origin[0]/MONITOR_WIDTH)*SCREEN_WIDTH + state_x*(state_width/MONITOR_WIDTH)*SCREEN_WIDTH
    y_click = (player_origin[1]/MONITOR_HEIGHT)*SCREEN_HEIGHT + (-1*state_y*(state_height/MONITOR_HEIGHT)*SCREEN_HEIGHT)
    click(int(x_click), int(y_click))
# set a certain state in the grid to a certain value to mark for example the type of unit in the state
def set_array_pos(grid, x, y, value, x_grid_size_in, y_grid_size_in):
    # make sure that we do not overwrite the state of our player character in the grid
    if not value == 1 and x == 0 and y == 0:
        return grid
    # resize grid in case we got uneven grid sizes
    if x_grid_size_in%2==1:
        x_grid_size = x_grid_size_in - 1
    else:
        x_grid_size = x_grid_size_in
    if y_grid_size_in%2==1:
        y_grid_size = y_grid_size_in -1
    else:
        y_grid_size = y_grid_size_in
    
    x_pos = x + int(x_grid_size/2)
    y_pos = y + int(y_grid_size/2)
    grid[x_pos][y_pos] = value    
    return grid

In [ ]:
## Intelligence
# TODO

## Main Routine

In [ ]:
sct = mss()
# determine the size of the area to observe (the game)
mon = {'top': 0, 'left': 0, 'width': SCREEN_WIDTH, 'height': SCREEN_HEIGHT}
# timer to see how fast the loop is running
last_time = time.time()
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      print('Loop took: {} seconds'.format(time.time()-last_time))
      last_time = time.time()
      #screen = cv2.resize(grab_screen(region=(0,40,1024,768)), (800,450))
      #screen = np.array(ImageGrab.grab(bbox=(0,40,1024,768)))
      # get screen recording and resize it to 800x450 pixels for output
      sct.get_pixels(mon)
      screen = Image.frombytes('RGB', (sct.width, sct.height), sct.image)
      screen = np.array(screen)
      screen = cv2.resize(screen, (MONITOR_WIDTH, MONITOR_HEIGHT))
      image_np = screen
      # ===================tensorflow code===============
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})                  
      
      # ====================================================  
      # Visualization of the results of a detection
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          min_score_thresh=0.25,
          line_thickness=4)

      #===================Player Character=======================
      gameover = False
      vayne_found = False;
      for i,b in enumerate(boxes[0]):
          if scores[0][i] >= 0.25:
              #Vaynefound makes sure that the whole grid + decision proces only executed once per detection in case of a multi detection
              if classes[0][i] == 1 and vayne_found == False:
                  vayne_found = True;
                  player_position = find_box_xy(boxes[0][i])
                  #click(int(player_position[1]*SCREEN_WIDTH), int(player_position[0]*SCREEN_HEIGHT))
                  #draw a circle where the character is
                  player_origin = [int(player_position[1]*MONITOR_WIDTH), int(player_position[0]*MONITOR_HEIGHT)]
                  cv2.circle(image_np, (player_origin[0], player_origin[1]), 2, (0, 0, 255), 2)
                  #calculate the origin of the center rectangle under the character
                  origin = [int(player_origin[0]+9)-int(player_position[2]*MONITOR_WIDTH/3), player_origin[1]-15,int(player_origin[0])+20, int(player_origin[1])+18]        
                  cv2.rectangle(image_np,(origin[0], origin[1]),(origin[2], origin[3]), (0, 255, 0), 1)
                  #create draw states
                  w = origin[2] - origin[0]
                  h = origin[3] - origin[1]
                  grid_x = int(SCREEN_WIDTH/w)
                  grid_y = int(SCREEN_HEIGHT/h)
                  dim = (grid_x,grid_y)
                  unit_grid = np.zeros(dim) 
                  #set the origin of the grid to be the player character represented by a 1
                  #the grid goes from -grid_x/2 to grid_x/2 because the player is the center and everything left and under it is negative
                  unit_grid = set_array_pos(unit_grid,0,0,1,grid_x,grid_y)
                  #=======DETECT OBJECTS=======
                  #1.Minions
                  for j,k in enumerate(boxes[0]):
                      if scores[0][j] >= 0.25:
                          if classes[0][j] == 2:
                              cur_minion_position = find_box_xy(boxes[0][j])
                              cur_minion_origin = [int(cur_minion_position[1]*MONITOR_WIDTH), int(cur_minion_position[0]*MONITOR_HEIGHT)]
                              cv2.circle(image_np, (cur_minion_origin[0], cur_minion_origin[1]), 2, (0, 0, 255), 2)
                              #Find in which state in the unit_grid the minion is
                              cur_minion_state = find_object_state(unit_grid, cur_minion_origin, player_origin)
                              unit_grid = set_array_pos(unit_grid,cur_minion_state[0],cur_minion_state[1],2,grid_x,grid_y)
                              
                    
                    
                  #=============================
                  #Visualize Objects
                  for x in range(len(unit_grid)):
                      for y in range(len(unit_grid[0])):
                          #cv2.rectangle(image_np,(origin[0]+w*x, origin[1]+h*y),(origin[2]+w*x, origin[3]+h*y), (0, 255, 0), 1)
                          if unit_grid[x][y] == 1:
                          #Paint player rectangle blue
                              cv2.rectangle(image_np,(origin[0]+w*x-int(grid_x/2)*w, origin[1]+h*y-int(grid_y/2)*h),(origin[2]+w*x-int(grid_x/2)*w, origin[3]+h*y-int(grid_y/2)*h), (0, 255, 0), 1)
                              cv2.rectangle(image_np,(origin[0]+2+w*x-int(grid_x/2)*w, origin[1]+2+h*y-int(grid_y/2)*h),(origin[2]-2+w*x-int(grid_x/2)*w, origin[3]-2+h*y-int(grid_y/2)*h), (0, 0, 255), 2)
                          elif unit_grid[x][y] == 2:
                              cv2.rectangle(image_np,(origin[0]+2+w*x-int(grid_x/2)*w, origin[1]+2+h*y-int(grid_y/2)*h),(origin[2]-2+w*x-int(grid_x/2)*w, origin[3]-2+h*y-int(grid_y/2)*h), (255, 0, 0), 2)
                          else:
                              # draw empty rectangles green
                              cv2.rectangle(image_np,(origin[0]+w*x-int(grid_x/2)*w, origin[1]+h*y-int(grid_y/2)*h),(origin[2]+w*x-int(grid_x/2)*w, origin[3]+h*y-int(grid_y/2)*h), (0, 255, 0), 1)
                  #=======Decision Making========
                
                  #=======Markov Process==========
                    
                  #=======Execute Click===========
                  #click_state(1,1,w,h, player_origin)
      # ==================OPEN CV visualization=================
      cv2.imshow('window',image_np)
      if (cv2.waitKey(25) & 0xFF == ord('q')):
          cv2.destroyAllWindows()
          break
      if gameover == True:
          print('gameover')
          cv2.destroyAllWindows()
          break;
